In [3]:
cfg = {
  "file_paths": {
    "preprocessed": "data/preprocessed",
    "embedded": "data/embedded",
    "predicted": "data/predicted"
  },
  "dataload": {
    "amplus": {
      "use_final_distribution": True,
      "use_torch": True,
      "prune_dist": 25
    },
    "dmgfull": {
      "use_final_distribution": True,
      "use_torch": True,
      "prune_dist": 25
    },
    "mdgenre": {
      "use_final_distribution": True,
      "use_torch": True,
      "prune_dist": 25
    },
    "dmg777k": {
      "use_final_distribution": True,
      "use_torch": True,
      "prune_dist": 25
    }
  },
  "preprocess": {
    "bin_numbers2": {
      "num_bins": 10
    },
    "append_weekday_and_month": {
      "neighborhood": 1
    },
    "delete_number_literals": {
      "neighborhood": 1
    },
    "do_nothing": {
      "nothingness": True
    },
    "delete_all_literals": {
      "neighborhood": 1
    }
  },
  "embed": {
    "RDF2Vec": {
      "embedding_name": "Word2Vec",
      "embedding_args": {
        "workers": 4,
        "epochs": 40
      },
      "walker_name": "RandomWalker",
      "walker_args": {
        "max_depth": 3,
        "max_walks": 500
      }
    },
    "RDF2Vec2": {
      "embedding_name": "Word2Vec",
      "embedding_args": {
        "workers": 4,
        "epochs": 40
      },
      "walker_name": "RandomWalker",
      "walker_args": {
        "max_depth": 2
      }
    },
    "TransE": {
      "optimizer": "Adam",
      "train_loop_type": "SLCWA",
      "train_loop_args": {
        "num_epochs": 40,
        "batch_size": 2048
      }
    },
    "complex": {
      "optimizer": "Adam",
      "train_loop_type": "SLCWA",
      "train_loop_args": {
        "num_epochs": 30,
        "batch_size": 1024
      }
    }
  },
  "evaluate": {
    "RandomForest": {
      "n_estimators": [
        10,
        20,
        40
      ],
      "max_depth": [
        3,
        5,
        10
      ],
      "cv": 10
    },
    "SVM": {
      "n_estimators": [
        10,
        20,
        40
      ],
      "max_depth": [
        3,
        5,
        10
      ],
      "cv": 10
    },
    "KNN": {
      "n_neighbors": [
        2,
        4,
        7,
        9,
        15
      ],
      "leaf_size": [
        10,
        20,
        30,
        50
      ]
    },
    "NB": {
      "alpha": [
        1,
        2,
        3
      ]
    }
  },
  "pipeline": {
    "dataload": "dmg777k",
    "preprocess": [
      "bin_numbers2",
      "delete_number_literals"
    ],
    "embed": "RDF2Vec2",
    "evaluate": [
      "SVM",
      "KNN"
    ]
  },
  "pipeline_schedule": {
    "dataload": [
      "dmg777k",
      "mdgenre"
    ],
    "preprocess": [
      [
        "bin_numbers2"
      ],
      [
        "bin_numbers2",
        "delete_all_literals"
      ],
      [
        "do_nothing"
      ],
      [
        "delete_all_literals"
      ],
      [
        "delete_all_but_numbers"
      ]
    ],
    "embed": [
      "TransE",
      "RDF2Vec",
      "RDF2Vec2"
    ],
    "iterations": 2
  }
}

In [4]:
def process_n_times(cfg) -> None:
    for dataload in cfg['pipeline_schedule']['dataload']:
        for preprocess in cfg['pipeline_schedule']['preprocess']:
            for embed in cfg['pipeline_schedule']['embed']:
                #for _ in range(cfg['pipeline_schedule']['iterations']):
                #    # TODO Logik einbauen um unnötige iterationen zu überspringen               
                #    cfg['pipeline']['dataload'] = dataload
                #    cfg['pipeline']['preprocess'] = preprocess
                #    cfg['pipeline']['embed'] = embed
                    

In [5]:
import os
import numpy as np
import re
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

#df = pd. 

print('pykeen file does not exist. Writing pykeen file...')
df = pd.DataFrame(columns=["full_name","f1_macro","f1_micro"])
        
dfs = []
for entry in os.listdir('../data/predicted/'):
    results = np.loadtxt(f'../data/predicted/{entry}', dtype=np.int32, delimiter=',')
    
    dfs.append(pd.DataFrame(
        [[
            entry,
            f1_score(results[0],results[1],average='micro'),
            f1_score(results[0],results[1],average='macro'),
        ]],
        columns=["full_name","f1_macro","f1_micro"]
    ))
df = pd.concat(dfs,ignore_index=True)
df['base_name']= df['full_name'].str.extract(r'(.*)\$[0-9]+\.csv')
df = df.groupby('base_name').mean().round(3)
#df['name'] = df.index
df = df.reset_index()
df['dataset']= df['base_name'].str.extract(r'(.*?)\+.*')
df['eval_method']= df['base_name'].str.extract(r'.*\$([A-Z]+)')
df['steps'] = df['base_name'].str.findall(r'\+(.*?)-')
df['steps'] = [', '.join(map(str, l)) for l in df['steps']]
df['embedder'] = df['base_name'].str.extract(r'\$(.*?)\$')
df = df.drop(columns='base_name')

#entries = {}
#for entry in os.listdir('../data/predicted/'):
#    if re.sub(r'\$[0-9]+\.csv','',entry) not in entries:
#        entries[re.sub(r'\$[0-9]+\.csv','',entry)] = []
    #if entry.split('+')[0]=='dmg777k':
#    entries[re.sub(r'\$[0-9]+\.csv','',entry)].append(np.loadtxt(f'../data/predicted/{entry}', dtype=np.int32, delimiter=','))

pykeen file does not exist. Writing pykeen file...


C:\Users\Noctris\AppData\Local\Temp\ipykernel_25008\1000925652.py:26: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.groupby('base_name').mean().round(3)


In [6]:
df

,f1_macro,f1_micro,dataset,eval_method,steps,embedder
0,0.582,0.366,dmg777k,KNN,bin_numbers2,RDF2Vec
1,0.631,0.534,dmg777k,SVM,bin_numbers2,RDF2Vec
2,0.619,0.473,dmg777k,KNN,bin_numbers2,RDF2Vec2
3,0.590,0.533,dmg777k,SVM,bin_numbers2,RDF2Vec2
4,0.493,0.246,dmg777k,KNN,bin_numbers2,TransE
5,0.523,0.246,dmg777k,SVM,bin_numbers2,TransE
6,0.588,0.387,dmg777k,KNN,"bin_numbers2, delete_all_literals",RDF2Vec
7,0.564,0.491,dmg777k,SVM,"bin_numbers2, delete_all_literals",RDF2Vec
8,0.587,0.442,dmg777k,KNN,"bin_numbers2, delete_all_literals",RDF2Vec2
9,0.563,0.489,dmg777k,SVM,"bin_numbers2, delete_all_literals",RDF2Vec2


In [20]:
for dataload in cfg['pipeline_schedule']['dataload']:
    for preprocess in cfg['pipeline_schedule']['preprocess']:
        preprocess_string =""
        for i in range(len(preprocess)):
            preprocess_string = preprocess_string + preprocess[i]
            if i < len(preprocess)-1:
                preprocess_string = preprocess_string +", "
        #preprocess_string = [', '.join(map(str, l)) for l in preprocess]
        for embed in cfg['pipeline_schedule']['embed']:
            #print(preprocess[0])
            #print(preprocess_string)
            iterations = cfg['pipeline_schedule']['iterations'] - len(df[(df["dataset"] == dataload)
                  & (df['embedder'] == embed) & (df['steps']== preprocess_string)])
            print(iterations)            
            #print(len(df[(df["dataset"] == dataload)
            #      & (df['embedder'] == embed) & (df['steps']== preprocess_string)]))


0
0
0
0
0
0
0
0
0
0
0
0
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2


In [21]:
df['steps'][0]

'bin_numbers2'

In [22]:
preprocess_string

'delete_all_but_numbers'